# Environment Setup

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/My\ Drive/data/*.zip .
!unzip /content/sudoku.zip
# !unzip /content/sudoku_test.zip

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Archive:  /content/sudoku.zip
  inflating: sudoku.csv              


In [22]:
!git clone https://github.com/cloughurd/drl-sudoku.git
!mv drl-sudoku/data/* .

Cloning into 'drl-sudoku'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 87 (delta 46), reused 33 (delta 11), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import librosa
import os
import gc
import wave
import struct
import math
import contextlib
from mido import MidiFile
from mido.messages.messages import Message
import mido
from typing import List
from tqdm import tqdm
import random
from torch.utils.data.sampler import SubsetRandomSampler


from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

# Create dataloader

In [0]:
def normalize_mono_grid(m):
  return (m / 9.0) - 0.5

In [0]:
from dataloader import *

# Define Model

In [0]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(self.shape)

In [0]:
class ToSudokuRange(nn.Module):
  def __init__(self):
    super(ToSudokuRange, self).__init__()
    self.sigmoid = nn.Sigmoid()
    self.net = lambda x: (9 * self.sigmoid(x)) + 0.5
  
  def forward(self, x):
    return self.net(x)

In [0]:
class SolverLayer(nn.Module):
  def __init__(self, in_filters:int, hidden_filters:int, out_filters:int):
    super(SolverLayer, self).__init__()

    if in_filters != out_filters:
      self.skip = nn.Conv2d(in_filters, out_filters, kernel_size=1)
    else:
      self.skip = nn.Identity()

    self.initial_normalization = nn.InstanceNorm2d(in_filters) # Normalize every individual game within its filters. Not positive this is a good idea... maybe use the 3d version?

    self.HorizontalDependencies = nn.Sequential(        
        nn.Conv2d(in_filters, hidden_filters, kernel_size=(9, 1)), # Output is 1, 9.
        nn.BatchNorm2d(hidden_filters),
        nn.LeakyReLU(),
        Reshape((-1, hidden_filters, 9)),
        # Make into a full board shape that can be recombined... Maybe?
        nn.Linear(9, 81),
        nn.BatchNorm1d(hidden_filters),
        nn.Dropout(),
        Reshape((-1, hidden_filters, 9, 9)),
        nn.LeakyReLU()
    )

    self.VerticalDependencies = nn.Sequential(
        nn.Conv2d(in_filters, hidden_filters, kernel_size=(1, 9)), # Output is 9, 1.
        nn.BatchNorm2d(hidden_filters),
        nn.LeakyReLU(),
        Reshape((-1, hidden_filters, 9)),
        # Make into a full board shape that can be recombined... Maybe?
        nn.Linear(9, 81), 
        nn.BatchNorm1d(hidden_filters),
        nn.Dropout(),
        Reshape((-1, hidden_filters, 9, 9)),
        nn.LeakyReLU()
    )

    self.QuadrantDependencies = nn.Sequential(
        nn.Conv2d(in_filters, hidden_filters, kernel_size=(3, 3), stride=3),
        nn.BatchNorm2d(hidden_filters),
        nn.LeakyReLU(),
        Reshape((-1, hidden_filters, 9)),
        # Make into a full board shape that can be recombined... Maybe?
        nn.Linear(9, 81), 
        nn.BatchNorm1d(hidden_filters),
        nn.Dropout(),
        Reshape((-1, hidden_filters, 9, 9)),
        nn.LeakyReLU()
    )

    self.Reduce = nn.Sequential(
        nn.Conv2d(hidden_filters * 3, hidden_filters*3, kernel_size=(3, 3), padding=1),
        nn.Conv2d(hidden_filters * 3, out_filters, kernel_size=(1, 1)), # Look at each cell only, without neighbors; neighbors have already been considered.
        nn.LeakyReLU()
    )

    self.Final = nn.LeakyReLU()

  def forward(self, x):
    skip = self.skip(x)
    x = self.initial_normalization(x)
    horizontal_result = self.HorizontalDependencies(x)
    vertical_result = self.VerticalDependencies(x)
    quadrant_result = self.QuadrantDependencies(x)

    combined = torch.cat((horizontal_result, vertical_result, quadrant_result), dim=1)

    reduced = self.Reduce(combined)

    residualized = reduced + skip
    return self.Final(residualized)
    


In [0]:
class MonoModel(nn.Module):
  def __init__(self, solver_depth:int):
    super(MonoModel, self).__init__()
    layer_numbers = [(3**(i // 9 + 2), 3**( (i+1) // 9 + 2)) for i in range(solver_depth - 2)]
    print(layer_numbers)
    reducer_layer = layer_numbers[-1][1]
    self.net = nn.Sequential(
        # Give a channel dimension
        Reshape((-1, 1, 9, 9)),
        SolverLayer(1, 9, 9),
        *[ SolverLayer(i, i, o) for i, o in layer_numbers],
        SolverLayer(reducer_layer, reducer_layer, 1),
        Reshape((-1, 9, 9)),
        ToSudokuRange()
    )

  def forward(self, x):
    return self.net(x)

In [0]:
def puzzle_exactifier(p):
  return torch.round(p)

def puzzle_masker(attempts, starting_puzzles):
  # print(attempts.size(), starting_puzzles.size())
  assert attempts.size() == starting_puzzles.size()
  num_filled_in_solution = torch.sum(starting_puzzles != 0).item()
  attempts = torch.where(starting_puzzles == 0, attempts, starting_puzzles)
  return (attempts, starting_puzzles, num_filled_in_solution)

def solved_accuracy(attempts, solutions, starting_puzzles):
  assert attempts.size() == solutions.size()
  masked_puzzle, _, _ = puzzle_masker(attempts, starting_puzzles)
  # print(masked_puzzle, solutions, starting_puzzles)
  num_puzzles = attempts.size(0)
  num_correct = (masked_puzzle.eq(solutions).sum(1).sum(1) == 9).sum().item()
  return num_correct / num_puzzles

def cell_accuracy(attempts, solutions, starting_puzzles):
  assert attempts.size() == solutions.size()
  masked_puzzle, _, num_filled = puzzle_masker(attempts, starting_puzzles)
  # print("\nAttempts:\n", attempts, "\nMasked\n", masked_puzzle, "\nInitial\n", starting_puzzles, "\nSolution\n", solutions)
  num_cells = attempts.numel()
  num_correct = masked_puzzle.eq(solutions).sum().item()
  num_guessed_correctly = num_correct - num_filled
  num_to_guess = num_cells - num_filled
  # print(f"--- {num_cells} - {num_correct} - {num_cells} - {num_filled} ---")
  return num_guessed_correctly / num_to_guess


# Training Loop

In [0]:
def save_out(model:MonoModel, **kargs):
  state = {
      "model": model.state_dict()
  }

  for k, v in kargs.items():
    state[k]=v
  
  torch.save(state, "/content/gdrive/My Drive/data/models/MonoModel.mod")

def train(model:MonoModel, criterion:torch.nn.modules.loss._Loss, optimizer:torch.optim.Optimizer, train_loader:DataLoader, valid_loader:DataLoader, num_epochs:int, valid_frequency:int=5, save_interval:int=1000):
  loop = tqdm(total=num_epochs * len(train_loader) + (num_epochs // valid_frequency) * len(valid_loader), position=0)

  # The loss of the last training and validation iteration, respectively.  
  train_loss = None
  valid_loss = None

  # The puzzle accuracy of the last training and validation itersions. ( # puzzles right / # puzzles total )
  train_accuracy = None
  valid_accuracy = None

  # The cell accuracy of the last training and validation iterations ( # filled cells right / # num fillable cells )
  train_inner_acc = None
  valid_inner_acc = None

  tot_training_losses = []
  tot_training_accuracies = []
  tot_training_cell_accuracies = []

  for e in range(num_epochs):

    training_losses = []
    training_accuracies = []
    training_cell_accuracies = []

    loop.set_description(f"[Training] Epoch: {e}. Loss: {valid_loss}/{train_loss}. Total Accuracy: {train_accuracy}/{valid_accuracy}. Inner Accuracy: {train_inner_acc}/{valid_inner_acc}")

    for i, (puzzle, solution) in enumerate(train_loader):
      puzzle, solution = puzzle.float().squeeze(1).cuda(async=False), (solution.float() + 1).cuda(async=False)
      initial_puzzle = puzzle.clone()

      optimizer.zero_grad()
      attempt = model(puzzle)
      loss = criterion(attempt, solution)

      loss.backward()
      optimizer.step()

      train_loss = loss.item()
      training_losses.append(train_loss)

      # Compute accuracies
      exactified = puzzle_exactifier(attempt)

      solve_accuracy = solved_accuracy(exactified, solution, initial_puzzle)
      c_acc = cell_accuracy(exactified, solution, initial_puzzle)

      train_accuracy = solve_accuracy
      training_accuracies.append(train_accuracy)

      train_inner_acc = c_acc
      training_cell_accuracies.append(train_inner_acc)

      loop.set_description(f"[Training] Epoch: {e}. Loss: {valid_loss}/{train_loss}. Total Accuracy: {valid_accuracy}/{train_accuracy}. Inner Accuracy: {train_inner_acc}/{valid_inner_acc}")
      loop.update()

      if i % save_interval == 0:
        tmp_losses = tot_training_losses + [training_losses]
        tmp_game_accs = tot_training_accuracies + [training_accuracies]
        tmp_cell_accs = tot_training_cell_accuracies + [training_cell_accuracies]
        save_out(model, train_loss=tmp_losses, train_game_accs=tmp_game_accs, train_cell_accs=tmp_cell_accs, epoch=e, iteration=i)
    tot_training_losses.append(training_losses)
    tot_training_accuracies.append(training_accuracies)
    tot_training_cell_accuracies.append(training_cell_accuracies)
    save_out(model, train_loss=tot_training_losses, train_game_accs=tot_training_accuracies, train_cell_accs=tot_training_cell_accuracies, epoch=e, iteration=-1)




In [0]:
model = MonoModel(solver_depth=21).cuda()
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()
train_loader = get_loader(root="/content/", batch_size=512, mono=True, train=True)
valid_loader = []
num_epochs = 5

In [0]:
num_epochs = 100

In [0]:
train(model, criterion, optimizer, train_loader, valid_loader, num_epochs, 1)

[Training] Epoch: 0. Loss: None/6.626322269439697. Total Accuracy: None/0.0. Inner Accuracy: 0.11238606394299727/None:   0%|          | 529/1757900 [01:29<105:25:18,  4.63it/s]

# TEST ACCURACY METRIC METHODS

In [0]:
j = torch.Tensor([[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]]])
k = torch.Tensor([[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]]])
l = torch.Tensor([[[2,2,2],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]]])

z = torch.zeros((3,3,3))
f = z.clone()
f[:,:,0] = k[:,:,0]
g = z.clone()
g[:,:,0] = l[:,:,0]

tensor([[[2., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]],

        [[1., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]],

        [[1., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]]])


In [0]:
s = torch.load("/content/gdrive/My Drive/data/models/MonoModel.mod")

In [0]:
s

In [13]:
s.keys()


dict_keys(['model', 'train_loss', 'train_game_accs', 'train_cell_accs', 'epoch', 'iteration'])

In [31]:
model.load_state_dict(s["model"])

<All keys matched successfully>

In [0]:
train_loader = get_loader(root="/content/", batch_size=2, mono=True, train=True)

In [0]:
x, y = next(iter(train_loader))

In [0]:
y = y + 1

In [23]:
print(x, "\n", y)

tensor([[[[1., 0., 2., 0., 0., 0., 0., 6., 0.],
          [6., 4., 0., 0., 1., 0., 7., 0., 0.],
          [0., 5., 0., 7., 0., 0., 0., 0., 0.],
          [0., 0., 5., 8., 7., 4., 0., 0., 6.],
          [0., 1., 0., 2., 0., 0., 3., 0., 0.],
          [2., 0., 0., 0., 0., 5., 0., 4., 0.],
          [7., 0., 0., 6., 5., 8., 4., 9., 3.],
          [5., 0., 4., 0., 0., 7., 0., 0., 2.],
          [3., 0., 6., 0., 2., 0., 8., 5., 7.]]],


        [[[0., 2., 0., 4., 0., 1., 0., 7., 0.],
          [0., 4., 7., 3., 2., 0., 0., 0., 0.],
          [5., 0., 6., 0., 8., 0., 0., 0., 4.],
          [7., 5., 0., 2., 3., 0., 6., 0., 1.],
          [0., 1., 4., 8., 6., 0., 2., 9., 5.],
          [8., 6., 0., 0., 0., 0., 4., 0., 0.],
          [4., 7., 3., 9., 0., 0., 0., 0., 8.],
          [0., 0., 0., 6., 0., 0., 0., 0., 0.],
          [0., 8., 0., 5., 0., 0., 0., 0., 0.]]]], dtype=torch.float64) 
 tensor([[[1., 7., 2., 9., 8., 3., 5., 6., 4.],
         [6., 4., 3., 5., 1., 2., 7., 8., 9.],
         [8.

In [32]:
print(puzzle_exactifier(model(x.float().cuda())), "\n", y)

tensor([[[4., 4., 4., 6., 5., 5., 5., 5., 5.],
         [5., 5., 5., 4., 4., 5., 6., 5., 5.],
         [6., 5., 5., 5., 5., 5., 4., 4., 5.],
         [5., 5., 5., 6., 6., 5., 4., 3., 5.],
         [5., 5., 5., 5., 5., 4., 5., 5., 7.],
         [5., 5., 5., 5., 5., 5., 5., 4., 7.],
         [6., 6., 6., 5., 5., 5., 4., 5., 3.],
         [5., 5., 5., 5., 6., 7., 4., 5., 4.],
         [5., 5., 5., 3., 4., 4., 7., 8., 5.]],

        [[4., 4., 5., 4., 4., 5., 8., 6., 5.],
         [5., 5., 6., 4., 5., 6., 5., 5., 4.],
         [5., 5., 6., 6., 6., 6., 4., 4., 3.],
         [5., 5., 5., 5., 6., 5., 5., 6., 4.],
         [5., 4., 4., 4., 4., 4., 5., 8., 6.],
         [7., 6., 6., 6., 5., 5., 3., 5., 3.],
         [5., 5., 5., 5., 5., 5., 5., 4., 7.],
         [5., 5., 5., 5., 6., 5., 5., 3., 5.],
         [5., 5., 5., 5., 5., 5., 5., 4., 7.]]], device='cuda:0',
       grad_fn=<RoundBackward>) 
 tensor([[[1., 7., 2., 9., 8., 3., 5., 6., 4.],
         [6., 4., 3., 5., 1., 2., 7., 8., 9.],
     

In [0]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [34]:
params

51688

In [60]:
model = MonoModel(solver_depth=21).cuda()
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

[(9, 9), (9, 9), (9, 9), (9, 9), (9, 9), (9, 9), (9, 9), (9, 9), (9, 27), (27, 27), (27, 27), (27, 27), (27, 27), (27, 27), (27, 27), (27, 27), (27, 27), (27, 81), (81, 81)]
2321264


In [0]:
l = get_loader("/content/", batch_size=1, cap_train=10)

In [0]:
x, y = next(iter(l))

In [0]:
y = y + 1
x,y = x.float().cuda(), y.float().cuda()

In [61]:
model(x)

tensor([[[4.9943, 5.6332, 4.9995, 5.1285, 4.9935, 4.9972, 5.4056, 4.9936,
          5.3876],
         [4.9999, 4.9950, 4.9978, 4.9994, 4.9979, 4.9993, 4.9945, 5.3312,
          5.2840],
         [4.9949, 4.9987, 4.9933, 4.9949, 5.3602, 4.9947, 4.9929, 5.1748,
          4.9967],
         [4.9952, 4.9925, 4.9921, 5.6491, 4.9951, 4.9961, 5.2364, 5.3954,
          4.9952],
         [5.2926, 4.9907, 4.9983, 4.9954, 4.9954, 4.9983, 5.1374, 4.9982,
          6.0181],
         [4.9972, 4.9985, 5.3440, 5.0613, 4.9937, 4.9898, 5.5038, 4.9964,
          5.0203],
         [5.3115, 4.9999, 5.6669, 5.3995, 4.9856, 4.9997, 5.0589, 4.9936,
          5.2659],
         [4.9947, 4.9945, 4.9941, 4.9943, 5.0100, 4.9895, 4.9884, 5.1754,
          4.9901],
         [4.9967, 4.9945, 5.0000, 5.1664, 5.0258, 4.9948, 5.0051, 4.9969,
          5.8408]]], device='cuda:0', grad_fn=<AddBackward0>)